In [1]:

!pip install datasets --quiet
!pip install rouge_score --quiet
!pip install pyarrow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [2]:

import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    BartTokenizer, BartForConditionalGeneration
)
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:1%]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
# T5
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

# BART
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [8]:
import time
import os
from google.api_core.exceptions import InternalServerError
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate


os.environ["GOOGLE_API_KEY"] = "***************"


def load_llm(model="gemini-1.5-pro"):
    if model == "gemini-1.5-pro":
        return ChatGoogleGenerativeAI(
            model="gemini-1.5-pro",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
    elif model == "gemini-1.5-flash":
        return ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2
        )
    else:
        raise ValueError("Invalid model name")


def get_prompt_template():
    return ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Write a concise summary of the following in {num_words} words:\n\n",
            ),
            ("human", "{context}")
        ]
    )


def summarize_text_with_retry(text, num_words=50, model="gemini-1.5-pro", retries=3):
    llm = load_llm(model)
    prompt = get_prompt_template()
    chain = prompt | llm
    attempt = 0
    while attempt < retries:
        try:
            result = chain.invoke({"context": text, "num_words": num_words})
            return result.content
        except InternalServerError as e:
            print(f"Server error: {e}. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)
            attempt += 1
    raise RuntimeError("Failed to summarize text after multiple attempts. Please try again later.")

if __name__ == "__main__":
    text = """LangChain offers various techniques for summarization such as MapReduce, Refine, Stuffing,
              and Map-Rerank. These methods cater to different document sizes and use cases, making
              LangChain highly versatile."""
    num_words = 30
    try:
        summary = summarize_text_with_retry(text, num_words=num_words, model="gemini-1.5-pro")
        print("Summary:", summary)
    except RuntimeError as e:
        print("Error:", e)


Server error: 500 Unable to submit request because the service is temporarily unavailable.. Retrying in 1 seconds...


Server error: 500 Unable to submit request because the service is temporarily unavailable.. Retrying in 2 seconds...


Server error: 500 Unable to submit request because the service is temporarily unavailable.. Retrying in 4 seconds...
Error: Failed to summarize text after multiple attempts. Please try again later.


In [9]:
def generate_summary_t5(text):
    """Generate summary using T5."""
    preprocessed_text = "summarize: " + text
    inputs = t5_tokenizer.encode(preprocessed_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def generate_summary_bart(text):
    """Generate summary using BART."""
    inputs = bart_tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], num_beams=4, max_length=150, min_length=40, length_penalty=2.0, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)



def calculate_rouge(reference, hypothesis):
    """Calculate ROUGE scores."""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return {
        "ROUGE-1": scores["rouge1"].fmeasure,
        "ROUGE-2": scores["rouge2"].fmeasure,
        "ROUGE-L": scores["rougeL"].fmeasure,
    }


def calculate_bleu(reference, hypothesis, weights=(0.25, 0.25, 0.25, 0.25)):
    """Calculate BLEU score with customizable N-grams."""
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(
        [reference_tokens],
        hypothesis_tokens,
        weights=weights,
        smoothing_function=smoothie
    )
    return score

In [10]:
def evaluate_summarization_models():
    for i, sample in enumerate(dataset):
        article = sample["article"]
        reference_summary = sample["highlights"]

        print("\n--- Sample Selected ---")
        print(f"**Original Article**:\n{article}\n")
        print(f"**Sample Summary**:\n{reference_summary}\n")


        t5_summary = generate_summary_t5(article)
        bart_summary = generate_summary_bart(article)
        llm_summary = summarize_text(article, num_words=50, model="gemini-1.5-flash")

        print("--- Generated Summaries ---")
        print(f"**T5 Summary**:\n{t5_summary}\n")
        print(f"**BART Summary**:\n{bart_summary}\n")
        print(f"**Google Gemini Summary**:\n{llm_summary}\n")

        for model_name, hypothesis in [("T5", t5_summary), ("BART", bart_summary), ("Google Gemini", llm_summary)]:
            rouge_scores = calculate_rouge(reference_summary, hypothesis)
            bleu_score = calculate_bleu(reference_summary, hypothesis)

            print(f"--- {model_name} Scores ---")
            print(f"ROUGE-1: {rouge_scores['ROUGE-1']:.4f}")
            print(f"ROUGE-2: {rouge_scores['ROUGE-2']:.4f}")
            print(f"ROUGE-L: {rouge_scores['ROUGE-L']:.4f}")
            print(f"BLEU: {bleu_score:.4f}")
            print("-" * 50)

        if i >= 1:
            break


if __name__ == "__main__":
    evaluate_summarization_models()


--- Sample Selected ---
**Original Article**:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad

In [13]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:1%]")

samples = dataset.select(range(3))

results = []

def evaluate_summarization_models():
    for i, sample in enumerate(samples):
        article = sample["article"]
        reference_summary = sample["highlights"]

        t5_summary = generate_summary_t5(article)
        bart_summary = generate_summary_bart(article)
        llm_summary = summarize_text(article, num_words=50, model="gemini-1.5-flash")

        for model_name, hypothesis in [
            ("T5", t5_summary),
            ("BART", bart_summary),
            ("Google Gemini", llm_summary),
        ]:
            rouge_scores = calculate_rouge(reference_summary, hypothesis)
            bleu_score = calculate_bleu(reference_summary, hypothesis)

            results.append({
                "Model Name": model_name,
                "Original Article": article,
                "Sample Summary": reference_summary,
                "Generated Summary": hypothesis,
                "ROUGE-1": rouge_scores["ROUGE-1"],
                "ROUGE-2": rouge_scores["ROUGE-2"],
                "ROUGE-L": rouge_scores["ROUGE-L"],
                "BLEU": bleu_score,
            })



In [14]:

def save_results_to_excel(results, file_name="summarization_results.xlsx"):
    df = pd.DataFrame(results)
    df.to_excel(file_name, index=False)
    print(f"Results exported to {file_name}")